# Подключение библиотек

In [16]:
#!pip install catboost

In [2]:
import numpy as np
import pandas as pd
import catboost
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных

In [4]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [20]:
#!7z x /content/drive/MyDrive/train.csv.7z

In [3]:
train_data_path = "./train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [21]:
#!7z x /content/drive/MyDrive/songs.csv.7z

In [4]:
songs_data_path = "./songs.csv"
songs = pd.read_csv(songs_data_path)
songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [22]:
#!7z x /content/drive/MyDrive/song_extra_info.csv.7z

In [5]:
song_extra_info_data_path = "./song_extra_info.csv"
song_extra_info = pd.read_csv(song_extra_info_data_path)
song_extra_info.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


In [23]:
#!7z x /content/drive/MyDrive/members.csv.7z

In [6]:
members_data_path = "./members.csv"
members = pd.read_csv(members_data_path)
members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


# Обработка данных

Для начала соединим данные в одну таблицу

In [7]:
train_data_with_song_extra_info = pd.merge(train_data, song_extra_info, on='song_id', how='left')
combined_data = pd.merge(train_data_with_song_extra_info, songs, on='song_id', how='left')
final_data = pd.merge(combined_data, members, on='msno', how='left')

In [8]:
final_data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,name,isrc,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,Good Grief,GBUM71602854,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,1,0,NaN,7,20120102,20171005
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,Lords of Cardboard,US3C69910183,284584.0,1259,Various Artists,NaN,NaN,52.0,13,24,female,9,20110525,20170911
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,Hip Hop Is Dead(Album Version (Edited)),USUM70618761,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,NaN,52.0,13,24,female,9,20110525,20170911
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,Disco Africa,GBUQH1000063,255512.0,1019,Soundway,Kwadwo Donkoh,NaN,-1.0,13,24,female,9,20110525,20170911
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,Sleep Without You,QM3E21606003,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,NaN,52.0,1,0,NaN,7,20120102,20171005


# Обучение Catboost.Classifier

Разделим датасет на обучающую и тестовую выборку

In [11]:
train_data, val_data = train_test_split(final_data, test_size = 0.2)
train_features = train_data.drop(['msno', 'song_id', 'target'], axis=1)
train_target = train_data['target']
val_features = val_data.drop(['msno', 'song_id', 'target'], axis=1)
val_target = val_data['target']

Раскодируем признаки

In [12]:
categorical_features = ['source_system_tab', 'source_screen_name', 'source_type', 'name', 'isrc', 'artist_name', 'composer', 'lyricist', 'gender', 'genre_ids']
for feature in categorical_features:
    train_features[feature] = train_features[feature].str.decode('utf-8')
    val_features[feature] = val_features[feature].str.decode('utf-8')

Обучим модель

In [17]:
model = catboost.CatBoostClassifier()
# model.fit(train_features, train_target)

# Применим обученную модель

In [15]:
predictions = model.predict(val_features)
ndcg = ndcg_score([val_data['target']], [predictions], k=20)
print(f"NDCG_20: {ndcg}")

NDCG_20: 0.6107589897123928


# Вывод

Применение Catboost.Classifier дало качество **0.61** согласно метрике ndcg_20
Существует больший простор для работы с данными, а также другие, более совершенные, методы ранжирования

Но сам бустинг является хорошим бейзлайном, от которого стоит отталкиваться